In [2]:
"""
Code for estimating population exposed to hazard (drougth).
"""
# Copyright 2021 Conservation International

import ee
import json

# The service account email address authorized by your Google contact.
# Set up a service account as described in the README.
EE_ACCOUNT = 'gef-ldmp-server@gef-ld-toolbox.iam.gserviceaccount.com'

# The private key associated with your service account in JSON format.
EE_PRIVATE_KEY_FILE = 'D:/Gabriel/CI/TrendsEarth/Git/GeePython_Codes/te_key.json'

EE_CREDENTIALS = ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)

#ee.Initialize(EE_CREDENTIALS)
ee.Initialize()

In [3]:
# Load region of interest
region = ee.Geometry(geojson)

# Filter region/country of interest
# countryName = ee.String('Guyana')
# country = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', countryName))

# Set year of interest
year = 2001

# Set spi lag
# spi_lag = 2
# spi_lag = 3
# spi_lag = 6
# spi_lag = 9
spi_lag = 12 
# spi_lag = 24

# Get spi image based on the lag
if (spi_lag == 1){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag1')}
if (spi_lag == 2){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag2')}
if (spi_lag == 3){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag3')}
if (spi_lag == 6){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag6')}
if (spi_lag == 9){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag9')}
if (spi_lag == 12){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag12')}
if (spi_lag == 24){spi_gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag24')}

# select bands for the year of interest out of the SPI image
spi_dec = spi_gpcc.select('spi_'+year+'_12')

# Define drought classes based on spi values
# 0 to 999 -> mild drought
#-1000 to -1499 -> moderate drought
#-1500 to -1.999 -> severe drought
#-2000 and less -> extreme drough
droughtClass = ee.Image(0)
    .where(spi_dec.lte(0).and(spi_dec.gt(-1000)),1)
    .where(spi_dec.lte(-1000).and(spi_dec.gt(-1500)), 2)
    .where(spi_dec.lte(-1500).and(spi_dec.gt(-2000)), 3)
    .where(spi_dec.lte(-2000), 4).rename(['Hazard']).aside(print, 'Hazard')

#Map.addLayer(droughtClass.updateMask(droughtClass.neq(-32768).clip(country)), {min:0, max:4, palette: ['E1E1E1','FFB935','FF58A0','D700D7', '730073']},'Hazard')
#Map.addLayer(droughtClass.clip(country), {min:0, max:4, palette: ['E1E1E1','FFB935','FF58A0','D700D7', '730073']},'Hazard')


# AGGREGATING GRIDDED POPULATION DATASETS

# Load the WorldPop image with annual population count as bands
wp_imgCol = ee.Image("users/geflanddegradation/toolbox_datasets/worldpop_ppp_2000_2020_1km_global")
#print('WorldPop: ', wp_imgCol.select('p'+year))

# Get WorldPop native projection
wpProj = ee.Image(wp_imgCol).projection();
#print('WorldPop scale:', wpProj.nominalScale());

# Get the projection at scale (GPCC) that will be used to aggregate WorldPop pixels 
gpcc = ee.Image('projects/trends_earth/spi/spi_GPCC_monthly_v2020_1971-2019_monthly_gamma_SPI_lag1')
gpccProj = gpcc.projection()
#print('GPCC scale: ', gpccProj.nominalScale())

# Filter WorldPop img to get band for the year of interest
wpYear = wp_imgCol
    .select('p'+year)             # select band
    .clip(country)                # clip band by the country outline
    .setDefaultProjection(wpProj) # assure filtered band has a set crs
#print('WorlPop Year: ', wpYear)

# When dealing with gridded population dataset, it is REQUIRED to aggregate by using ```reduceResolution()```: https://spatialthoughts.com/2021/05/13/aggregating-population-data-gee/
#  use reduceResolution to aggregate worldpop to the CHIRPS cell size and grid

# Aggregate wp population data pixels to 30km
wpAgg30km = wpYear.reduceResolution({
    'reducer': ee.Reducer.sum().unweighted(),  # apply an unweighted sum reducer
    'maxPixels': 1024
  }).reproject({
    'crs':gpccProj # reproject to the gpcc scale == 27,829.87 meters
  })
#print(wpAgg30km)

# Summarize population count at gpcc scale
stats30km = wpAgg30km.reduceRegion({
    'reducer': ee.Reducer.sum(),
    'geometry': country,
    'scale': gpccProj.nominalScale(),
    'maxPixels': 1e13,
    'tileScale': 16
})
#print('Total Country Pop at 30km: ', stats30km.get('p'+year))

# Reassign the variable with the 30km aggregated total population count for the given country
stats30km = stats30km.get('p'+year)
#print('Total Country Pop at 30km: ', stats30km)

# Summarize population count at wp original scale
stats1km = wpYear.reduceRegion({
    'reducer': ee.Reducer.sum(),
    'geometry': country,
    'scale': wpProj.nominalScale(),
    'maxPixels': 1e13,
    'tileScale': 16
})
#print('Total Country Pop at 1km: ', stats1km.get('p'+year))

# Get total population count for the region/country of interest
stats1km = stats1km.get('p'+year)
#print('Total Country Pop at 1km: ', stats1km)

# Add a band representing drought intensity classes to the WorldPop image aggregated to the GPCC scale
wp30km_droughts = wpAgg30km.addBands(droughtClass)
#print('wp30km_droughts: ', wp30km_droughts)

# Vectorize the droughts raster so we have polygons representing each drought class
hazard = droughtClass.reduceToVectors({
    'geometry': country,
    'crs':droughtClass.projection(),
    'scale': gpccProj.nominalScale(),
    'geometryType': 'polygon',
    'eightConnected': true,
    'labelProperty': 'Hazard',
  #'bestEffort': true
})

# Create New Feature Collection based on drought class
maxError = 0.01;
# First get a list of all posible drought values
classes = ee.List(hazard.aggregate_array('Hazard')).distinct().flatten().sort().aside(print, 'Array');
# Make a feature the union of all features having the same name
hazardUnion = ee.FeatureCollection(classes.map(function(name){
    tempFC = hazard.filter(ee.Filter.eq('Hazard', name));
    unionFC = tempFC.union(maxError); # specifying a max error overcomes issues with features of diff projection
# Cast the featureCollection (output union()) to a single feature
    return ee.Feature(unionFC.first()).set('Hazard', name);
}));

#print(hazardUnion, 'Union')

#Map.addLayer(hazardUnion, {}, 'Hazard Polygons', false)

#Grouping Approach

# Use reduceRegion() to get the a dictonary with the hazard classes and respective population summarized - Exposure
exposureDict = wp30km_droughts.reduceRegion({
    'reducer': ee.Reducer.sum().group({
    'groupField': 1,
    'groupName': 'Hazard',
  }),
    'geometry': country,
    'scale': gpccProj.nominalScale(), 
    'maxPixels': 1e13,
    'tileScale': 16
})
#print('Exposure to Drought: ', exposureDict)

# Create a FeatureCollection representing Exposure classes and respective population exposure
groups = ee.List(exposureDict.get('groups'));

fc_groups = ee.FeatureCollection(groups.map(function(obj) {
    objct = ee.Dictionary(obj)
    name = ee.String(objct.get('Hazard'))
    summed = ee.List(objct.get('sum'))
    return ee.Feature(null, {'Hazard':name, 'sum':summed})
}))

# Bring back the geometries to FC : inner join dictionary with Exposure to FC with Multipolygons

innerJoin = ee.Join.inner();

filterTimeEq = ee.Filter.equals({
    leftField: 'system:index',
    rightField: 'system:index'
});

innerJoined = innerJoin.apply(hazardUnion, fc_groups, filterTimeEq);

# Function to structure the FC representing Exposure

so3_l2_exposure = innerJoined.map(function(pair) {
    f1 = ee.Feature(pair.get('primary'));
    f2 = ee.Feature(pair.get('secondary'));
    return f1.set(f2.toDictionary());
})
#print('SO3 L2 Exposure: ', so3_l2_exposure)

# create new column with proportion 
addProp = function(feature) {
    return feature.set({'Population_total': feature.getNumber('sum'), 'Percentage': feature.getNumber('sum').divide(stats30km).multiply(100)});
};
# Map the proportion getting function over the FeatureCollection.
so3_l2_exposure = so3_l2_exposure.map(addProp); 
so3_l2_exposure = so3_l2_exposure.set({'Country': countryName,'Year': year, 'Sources': ['GPCC', 'WorldPop', 'Trends.Earth']})

# Generic Function to remove a property from a feature
removeProperty = function(feat, property) {
    properties = feat.propertyNames()
    selectProperties = properties.filter(ee.Filter.neq('item', property))
    return feat.select(selectProperties)
}

# Remove property 'sum' in each feature
so3_l2_exposure = so3_l2_exposure.map(function(feat) {
    return removeProperty(feat, 'sum')
})

#print(so3_l2_exposure, 'SO3 L2 Exposure w/ Proportion')

# Export.table.toDrive({
#   'collection':so3_l2_exposure,
#   'description':'PopExpDrought_'+countryName+'_year',
#   'folder': 'SO3',
#   'fileFormat': 'SHP'
#   })


SyntaxError: invalid syntax (<ipython-input-3-25ff99ee759a>, line 20)